# 3. Training Pipeline

## 3.1. Setup

### 3.1.1. Import Libraries

In [ ]:
# Standard imports
import os
from pathlib import Path
import sys
import json
import time
from datetime import date, datetime, timedelta
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore", module="IPython")
warnings.filterwarnings("ignore", category=DeprecationWarning)

#  Establish project root directory
def find_project_root(start: Path):
    for parent in [start] + list(start.parents):
        if (parent / "pyproject.toml").exists():
            return parent
    return start

root_dir = find_project_root(Path().absolute())
print("Project root dir:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# Third-party imports
import requests
import pandas as pd
import numpy as np
import great_expectations as gx
import hopsworks
from urllib3.exceptions import ProtocolError
from requests.exceptions import ConnectionError, Timeout, RequestException
from confluent_kafka import KafkaException
from hsfs.client.exceptions import RestAPIError
from collections import defaultdict
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from scipy.spatial.distance import cdist

#  Project imports
from utils import cleaning, config, feature_engineering, fetchers, hopsworks_admin, incremental, metadata, visualization

today = datetime.today().date()


Project root dir: c:\Users\krist\Documents\GitHub\pm25


### 3.1.2. Load Settings and Initialize Hopsworks Connection 

In [ ]:

def detect_environment():
    if (
        "HOPSWORKS_JOB_ID" in os.environ
        or "HOPSWORKS_PROJECT_ID" in os.environ
        or "HOPSWORKS_JOB_NAME" in os.environ
    ):
        return "job"

    cwd = os.getcwd()
    if cwd.startswith("/hopsfs/Jupyter"):
        return "jupyter"

    return "local"

env = detect_environment()
print(f"Detected environment: {env}")

# Load secrets based on environment

if env in ("job", "jupyter"):
    project = hopsworks.login()
    secrets_api = hopsworks.get_secrets_api()

    for key in ["HOPSWORKS_API_KEY", "AQICN_API_KEY", "GH_PAT", "GH_USERNAME"]:
        os.environ[key] = secrets_api.get_secret(key).value

else:
    load_dotenv()

# Load Pydantic settings

settings = config.HopsworksSettings()

HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()
GITHUB_USERNAME = settings.GH_USERNAME.get_secret_value()

# Login to Hopsworks using the API key

project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

print("Environment initialized and Hopsworks connected!")


💻 Running locally
HopsworksSettings initialized!
2026-01-28 09:59:30,531 INFO: Closing external client and cleaning up certificates.
Connection closed.
2026-01-28 09:59:30,536 INFO: Initializing external client
2026-01-28 09:59:30,536 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-28 09:59:32,026 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279184


### 3.1.3. Repository Management

In [33]:
repo_dir = hopsworks_admin.clone_or_update_repo(GITHUB_USERNAME)
os.chdir(repo_dir)

💻 Already in git repository at c:\Users\krist\Documents\GitHub\pm25


### 3.1.4. Configure API Keys and Secrets

In [34]:
secrets = hopsworks.get_secrets_api()

try:
    secrets.get_secret("AQICN_API_KEY")
except:
    secrets.create_secret("AQICN_API_KEY", settings.AQICN_API_KEY.get_secret_value())

### 3.1.5. Get Model Registry

In [35]:
mr = project.get_model_registry()

## 3.2. Load Feature Groups & Metadata

In [36]:
air_quality_fg, weather_fg = hopsworks_admin.create_feature_groups(fs)

# Load data from air_quality feature group
aq_data = air_quality_fg.read()

if len(aq_data) == 0:
    print("⚠️ No air quality data found. Run pipeline 1 (backfill) first.")
    sys.exit(1)

# Extract metadata and set sensor_id as index for lookups
aq_metadata = aq_data[["sensor_id", "latitude", "longitude", "city", "street", "country"]].drop_duplicates(subset=["sensor_id"]).set_index("sensor_id")

# Build sensor location dictionary
sensor_locations = {}
for sensor_id, row in aq_metadata.iterrows():
    sensor_locations[sensor_id] = (
        row["latitude"], 
        row["longitude"], 
        row["city"], 
        row["street"], 
        row["country"]
    )
print(f"📍 Loaded locations for {len(sensor_locations)} sensors")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (16.01s) 
📍 Loaded locations for 103 sensors


## 3.3. Create Additional Feature Views
Create multiple feature views with different feature combinations (baseline, rolling windows, lagged features, nearby sensors, complete) for model comparison.

In [37]:
# Select features for training data
baseline_features = air_quality_fg.select(["pm25", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])

baseline_feature_view = fs.get_or_create_feature_view(
    name="air_quality_baseline_fv",
    description="Weather features for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=baseline_features,
)

rolling_features = air_quality_fg.select(["pm25", "pm25_rolling_3d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
rolling_feature_view = fs.get_or_create_feature_view(
    name="air_quality_rolling_fv",
    description="Weather features, PM2.5 rolling window (3d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=rolling_features,
)

nearby_features = air_quality_fg.select(["pm25", "pm25_nearby_avg", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
nearby_feature_view = fs.get_or_create_feature_view(
    name="air_quality_nearby_fv",
    description="Weather features, PM2.5 nearby average (1d lag, 3 sensors) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=nearby_features,
)

lagged_1d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
lagged_1d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_1d_fv",
    description="Weather features, PM2.5 lags (1d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_1d_features,
)

lagged_2d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "pm25_lag_2d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
lagged_2d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_2d_fv",
    description="Weather features, PM2.5 lags (1d, 2d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_2d_features,
)

lagged_3d_features = air_quality_fg.select(["pm25", "pm25_lag_1d", "pm25_lag_2d", "pm25_lag_3d", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
lagged_3d_feature_view = fs.get_or_create_feature_view(
    name="air_quality_lagged_3d_fv",
    description="Weather features, PM2.5 lags (1d, 2d, 3d) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=lagged_3d_features,
)

complete_features = air_quality_fg.select(["pm25", "pm25_rolling_3d", "pm25_lag_1d", "pm25_lag_2d", "pm25_lag_3d", "pm25_nearby_avg", "date", "sensor_id"]).join(
    weather_fg.select_features(), on=["sensor_id", "date"])
complete_feature_view = fs.get_or_create_feature_view(
    name="air_quality_complete_fv",
    description="Weather features, PM2.5 rolling window (3d), and PM2.5 lags (1d, 2d, 3d), and PM2.5 nearby average (1d lag, 3 sensors) for PM2.5 prediction",
    version=1,
    labels=["pm25"],
    query=complete_features,
)

2026-01-28 09:59:52,308 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-28 09:59:54,035 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-28 09:59:55,710 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-28 09:59:57,432 INFO: Using ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant'] from feature group `weather` as features for the query. To include primary key and event time use `select_all`.
2026-01-28 09:59

## 3.4. Training Setup

### 3.4.1. Define Training Directory

In [38]:
model_dir = f"{root_dir}/models"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
    
print(f"Model directory: {model_dir}")

Model directory: c:\Users\krist\Documents\GitHub\pm25/models


### 3.4.2. Define Model Hyperparameters

In [39]:
TARGET = "pm25"

TRAIN_RATIO = 0.8
MIN_ROWS = 10
MIN_TEST_ROWS = 2

EXCLUDE_COLS = [
    "pm25","date","sensor_id","city","street","country",
    "latitude","longitude","aqicn_url"
]

N_RESTARTS = 5
BASE_SEED = 165439

xgb_params = {
    "n_estimators": 100,
    "learning_rate": 0.05,
}

### 3.4.3. Initialize Containers for Results

In [40]:
# Prepare containers to store models, predictions, and results
models = defaultdict(dict)
y_preds = defaultdict(dict)
results = []

# Define feature view dictionary for iteration
feature_views = {
    "baseline": baseline_feature_view,
    "rolling": rolling_feature_view,
    "nearby": nearby_feature_view,
    "lagged_1d": lagged_1d_feature_view,
    "lagged_2d": lagged_2d_feature_view,
    "lagged_3d": lagged_3d_feature_view,
    "complete": complete_feature_view,
}

## 3.5. Training Loop
Train XGBoost models for each feature combination and sensor, run 5 iterations per configuration, select best model based on R2 score, and store results.

### 3.5.1. Load Feature View Data

In [41]:
feature_data_cache = {}
total_views = len(feature_views)

for i, (feature_name, feature_view) in enumerate(feature_views.items(), start=1):
    print(f"Reading ({i}/{total_views}): {feature_name}")

    df = feature_view.query.read()
    df["date"] = pd.to_datetime(df["date"]).dt.tz_localize(None)
    feature_data_cache[feature_name] = df

    print(f"    ✔ Loaded {len(df):,} rows")

print(f"\n✅ Loaded all {total_views} feature views")

Reading (1/7): baseline
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.98s) 
    ✔ Loaded 165,668 rows
Reading (2/7): rolling
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.30s) 
    ✔ Loaded 165,668 rows
Reading (3/7): nearby
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.65s) 
    ✔ Loaded 165,668 rows
Reading (4/7): lagged_1d
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.49s) 
    ✔ Loaded 165,668 rows
Reading (5/7): lagged_2d
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.71s) 
    ✔ Loaded 165,668 rows
Reading (6/7): lagged_3d
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (13.63s) 
    ✔ Loaded 165,668 rows
Reading (7/7): complete
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (20.35s) 
    ✔ Loaded 165,668 rows

✅ Loaded all 7 feature views


### 3.5.2. Build Task List

In [42]:
tasks = []
total_views = len(feature_data_cache)

print(f"Building task list from {total_views} feature views:")

for i, (feature_name, df) in enumerate(feature_data_cache.items(), start=1):
    sensor_ids = df["sensor_id"].unique()
    count = len(sensor_ids)

    print(f"[{i}/{total_views}] {feature_name}: {count} sensors")

    for sensor_id in sensor_ids:
        tasks.append((feature_name, sensor_id))

print(f"\n✅ {len(tasks):,} total training tasks")

Building task list from 7 feature views:
[1/7] baseline: 103 sensors
[2/7] rolling: 103 sensors
[3/7] nearby: 103 sensors
[4/7] lagged_1d: 103 sensors
[5/7] lagged_2d: 103 sensors
[6/7] lagged_3d: 103 sensors
[7/7] complete: 103 sensors

✅ 721 total training tasks


### 3.5.3. Training Loop

In [43]:
total = len(tasks)

for idx, (feature_name, sensor_id) in enumerate(tasks, start=1):
    df = feature_data_cache[feature_name]
    df = df[df["sensor_id"] == sensor_id].dropna(subset=[TARGET])

    if len(df) < MIN_ROWS:
        continue

    feature_cols = [c for c in df.columns if c not in EXCLUDE_COLS]

    train_size = int(TRAIN_RATIO * len(df))
    train_df = df.iloc[:train_size]
    test_df = df.iloc[train_size:]

    if len(test_df) < MIN_TEST_ROWS:
        continue

    X_train = train_df[feature_cols]
    y_train = train_df[TARGET]
    X_test = test_df[feature_cols]
    y_test = test_df[TARGET]

    best_model = None
    best_r2 = -1e9
    best_mse = 1e9

    for i in range(N_RESTARTS):
        model = XGBRegressor(
            n_estimators=xgb_params["n_estimators"],
            learning_rate=xgb_params["learning_rate"],
            random_state=BASE_SEED*i
        )
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        r2 = r2_score(y_test, pred)
        mse = mean_squared_error(y_test, pred)

        if r2 > best_r2:
            best_r2 = r2
            best_mse = mse
            best_model = model

    # Store results
    models.setdefault(feature_name, {})[sensor_id] = best_model
    y_preds.setdefault(feature_name, {})[sensor_id] = best_model.predict(X_test)
    results.append({
        "feature_name": feature_name,
        "sensor_id": sensor_id,
        "R2": best_r2,
        "MSE": best_mse,
        "train_size": len(train_df),
        "test_size": len(test_df),
    })

    if idx % 10 == 0 or idx == total:
        print(f"[{idx}/{total}] Trained {feature_name} / sensor {sensor_id}: R²={best_r2:.3f}, MSE={best_mse:.2f}")

print(f"\n✅ Training complete: {len(results)} models trained")

[10/721] Trained baseline / sensor 59593: R²=0.031, MSE=5.16
[20/721] Trained baseline / sensor 60838: R²=0.041, MSE=97.27
[30/721] Trained baseline / sensor 62566: R²=0.147, MSE=10.94
[40/721] Trained baseline / sensor 65707: R²=0.020, MSE=15.46
[50/721] Trained baseline / sensor 79750: R²=0.055, MSE=4.62
[60/721] Trained baseline / sensor 89584: R²=-0.032, MSE=18.37
[70/721] Trained baseline / sensor 122302: R²=-0.107, MSE=6.78
[80/721] Trained baseline / sensor 196735: R²=-0.069, MSE=3.66
[90/721] Trained baseline / sensor 404209: R²=-0.005, MSE=9.26
[100/721] Trained baseline / sensor 497266: R²=0.360, MSE=4.81
[110/721] Trained rolling / sensor 68167: R²=-3.470, MSE=90.27
[120/721] Trained rolling / sensor 61861: R²=0.225, MSE=3.75
[130/721] Trained rolling / sensor 65104: R²=0.370, MSE=0.69
[140/721] Trained rolling / sensor 58921: R²=0.907, MSE=0.04
[150/721] Trained rolling / sensor 88876: R²=0.260, MSE=11.02
[160/721] Trained rolling / sensor 59650: R²=0.040, MSE=8.99
[170/721

## 3.6. Model Selection

### 3.6.1. Identify Best Model per Sensor

In [44]:
results_df = pd.DataFrame(results)

best_models = results_df.loc[results_df.groupby("sensor_id")["R2"].idxmax()]
best_models = best_models.set_index("sensor_id")

print(f"Identified best models for {len(best_models)} sensors:\n")

summary = best_models[["feature_name", "R2", "MSE"]].sort_index()

print(summary.to_string())


Identified best models for 103 sensors:

          feature_name        R2         MSE
sensor_id                                   
57421        lagged_1d  0.405327    5.429496
58666        lagged_1d  0.202183    4.358064
58909        lagged_1d  0.369805    4.138953
58912        lagged_1d  0.319515    5.742841
58921         complete  0.919423    0.043539
59095        lagged_1d  0.322574   11.292411
59356           nearby  0.060369   12.561014
59410        lagged_3d  0.278112    0.249105
59497        lagged_1d  0.353158    6.069523
59593        lagged_3d  0.218126    4.166982
59650        lagged_2d  0.327375    5.599234
59656        lagged_3d  0.058061   68.517111
59887        lagged_1d  0.248315   10.020959
59893        lagged_1d  0.459874    2.417701
59899        lagged_1d  0.364586    7.362497
60073        lagged_2d  0.466981    2.714573
60076        lagged_1d  0.408150   12.842981
60535        lagged_1d  0.198454   17.757584
60541        lagged_2d  0.257535   11.924551
60838         

### 3.6.2. Load Feature View Data

In [45]:
total_views = len(feature_views)
print(f"Loading {total_views} feature views...\n")

cached_feature_data = {}

for i, (name, fv) in enumerate(feature_views.items(), start=1):
    print(f"[{i}/{total_views}] Reading feature view: {name}...")
    df_cached = fv.query.read()
    cached_feature_data[name] = df_cached
    print(f"    ✔ Loaded {len(df_cached):,} rows")

print("\nNormalizing date columns...\n")

for name, df_cached in cached_feature_data.items():
    df_cached["date"] = pd.to_datetime(df_cached["date"]).dt.tz_localize(None)
    print(f"    ✔ Normalized dates for '{name}'")

print("\nAll feature views loaded and normalized")


Loading 7 feature views...

[1/7] Reading feature view: baseline...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.89s) 
    ✔ Loaded 165,668 rows
[2/7] Reading feature view: rolling...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.81s) 
    ✔ Loaded 165,668 rows
[3/7] Reading feature view: nearby...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.77s) 
    ✔ Loaded 165,668 rows
[4/7] Reading feature view: lagged_1d...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.46s) 
    ✔ Loaded 165,668 rows
[5/7] Reading feature view: lagged_2d...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.74s) 
    ✔ Loaded 165,668 rows
[6/7] Reading feature view: lagged_3d...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.91s) 
    ✔ Loaded 165,668 rows
[7/7] Reading feature view: complete...
Finished: Reading da

### 3.9.3. Generate Predictions

In [46]:
all_test_data = []
total_sensors = len(best_models)

print(f"Processing {total_sensors} sensors...\n")

for idx, (_, row) in enumerate(best_models.iterrows(), start=1):
    sensor_id = row.name
    best_feature = row["feature_name"]

    status = []  # collect short status flags

    # --- Save model + feature importance ---
    sensor_dir = f"{model_dir}/{sensor_id}"
    images_dir = f"{sensor_dir}/images"
    os.makedirs(images_dir, exist_ok=True)

    best_model = models[best_feature][sensor_id]
    best_model.save_model(f"{sensor_dir}/model.json")
    plot_importance(best_model)
    plt.savefig(f"{images_dir}/feature_importance.png")
    plt.close()
    status.append("model+plot")

    # --- Load cached feature view data ---
    df = cached_feature_data[best_feature]
    df = df[df["sensor_id"] == sensor_id].copy()
    df = df.sort_values("date").reset_index(drop=True)
    df = df.dropna(subset=["pm25"])

    feature_cols = [
        c for c in df.columns
        if c not in ["pm25", "date", "sensor_id", "city", "street",
                     "country", "latitude", "longitude", "aqicn_url"]
    ]

    df_clean = df.copy()
    if len(df_clean) < 10:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (too few rows)")
        continue

    # --- Full predictions ---
    X_full = df_clean[feature_cols]
    df_clean["predicted_pm25"] = best_model.predict(X_full)
    df_clean["best_model"] = best_feature
    status.append("predictions")

    # --- Hindcast window ---
    cutoff_date = pd.Timestamp.now() - pd.DateOffset(months=18)
    df_hindcast = df_clean[df_clean["date"] >= cutoff_date].copy()

    if len(df_hindcast) == 0:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (no recent data)")
        continue
    status.append("hindcast")
  
    # --- Metadata ---
    if sensor_id in aq_metadata.index:
        df_clean["latitude"] = aq_metadata.at[sensor_id, "latitude"]
        df_clean["longitude"] = aq_metadata.at[sensor_id, "longitude"]
        df_clean["city"] = aq_metadata.at[sensor_id, "city"]
        df_clean["street"] = aq_metadata.at[sensor_id, "street"]
        df_clean["sensor_id"] = sensor_id
        status.append("metadata")
    else:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (no metadata)")
        continue

    # --- Append final data ---
    all_test_data.append(
        df_clean[[
            "date", "sensor_id", "pm25", "predicted_pm25",
            "latitude", "longitude", "city", "street", "best_model"
        ]]
    )
    status.append("saved")

    # --- One-line summary ---
    print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: " + ", ".join(status))

print(f"\nDone. Successfully processed {len(all_test_data)} sensors.")

Processing 103 sensors...

[1/103] Sensor 57421: model+plot, predictions, hindcast, metadata, saved
[2/103] Sensor 58666: model+plot, predictions, hindcast, metadata, saved
[3/103] Sensor 58909: model+plot, predictions, hindcast, metadata, saved
[4/103] Sensor 58912: skipped (no recent data)
[5/103] Sensor 58921: model+plot, predictions, hindcast, metadata, saved
[6/103] Sensor 59095: model+plot, predictions, hindcast, metadata, saved
[7/103] Sensor 59356: model+plot, predictions, hindcast, metadata, saved
[8/103] Sensor 59410: model+plot, predictions, hindcast, metadata, saved
[9/103] Sensor 59497: model+plot, predictions, hindcast, metadata, saved
[10/103] Sensor 59593: model+plot, predictions, hindcast, metadata, saved
[11/103] Sensor 59650: model+plot, predictions, hindcast, metadata, saved
[12/103] Sensor 59656: model+plot, predictions, hindcast, metadata, saved
[13/103] Sensor 59887: model+plot, predictions, hindcast, metadata, saved
[14/103] Sensor 59893: model+plot, predictions

In [47]:
# --- DIAGNOSTIC: Check data coverage by year ---
print("\n📊 DATA COVERAGE DIAGNOSTIC")
print("="*60)

for feature_name, df_cached in cached_feature_data.items():
    print(f"\n{feature_name}:")
    
    # Check one sensor as example
    sample_sensor = df_cached["sensor_id"].iloc[0]
    sensor_data = df_cached[df_cached["sensor_id"] == sample_sensor].copy()
    sensor_data["year"] = pd.to_datetime(sensor_data["date"]).dt.year
    
    print(f"  Sensor {sample_sensor}:")
    print(f"  Total rows: {len(sensor_data)}")
    print(f"  Date range: {sensor_data['date'].min()} to {sensor_data['date'].max()}")
    
    # Check pm25 by year
    yearly_stats = sensor_data.groupby("year").agg({
        "pm25": ["count", lambda x: x.isna().sum()]
    })
    yearly_stats.columns = ["Total Rows", "NaN Count"]
    yearly_stats["Non-NaN"] = yearly_stats["Total Rows"] - yearly_stats["NaN Count"]
    print("\n  PM2.5 availability by year:")
    print(yearly_stats)
    
    # After dropna
    sensor_data_clean = sensor_data.dropna(subset=["pm25"])
    if len(sensor_data_clean) > 0:
        print(f"\n  After dropna(subset=['pm25']):")
        print(f"    Remaining rows: {len(sensor_data_clean)}")
        print(f"    Date range: {sensor_data_clean['date'].min()} to {sensor_data_clean['date'].max()}")
        print(f"    Years present: {sorted(sensor_data_clean['year'].unique())}")
    else:
        print(f"\n  ⚠️ NO DATA LEFT after dropna!")
    
    break  # Only check first feature view for now

print("\n" + "="*60)


📊 DATA COVERAGE DIAGNOSTIC

baseline:
  Sensor 57421:
  Total rows: 2062
  Date range: 2019-12-09 00:00:00 to 2026-01-14 00:00:00

  PM2.5 availability by year:
      Total Rows  NaN Count  Non-NaN
year                                
2019          23          0       23
2020         359          0      359
2021         267          0      267
2022         363          0      363
2023         365          0      365
2024         347          0      347
2025         334          0      334
2026           4          0        4

  After dropna(subset=['pm25']):
    Remaining rows: 2062
    Date range: 2019-12-09 00:00:00 to 2026-01-14 00:00:00
    Years present: [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]



### 3.6.3. Preparation for Visualization and Registration

In [48]:
df = pd.concat(all_test_data, ignore_index=True) if all_test_data else pd.DataFrame()
df = df.sort_values(by=["date"])
df_by_sensor = {sid: g.copy() for sid, g in df.groupby("sensor_id")}

HINDCAST_MONTHS = 18

## 3.7. Visualization
Generate Plots and Hindcats Data

### 3.7.1. Insert Predictions to Feature Group

In [49]:
# Create or get monitoring feature group
monitor_fg = fs.get_or_create_feature_group(
    name="aq_predictions",
    description="Air Quality prediction monitoring from training",
    version=1,
    primary_key=["sensor_id", "date", "days_before_forecast_day"],
    event_time="date",
)

monitoring_predictions = []

print(f"\nPreparing predictions with features for {len(all_test_data)} sensors...\n")

for sensor_df in all_test_data:
    if sensor_df.empty:
        continue
    
    sensor_id = sensor_df["sensor_id"].iloc[0]
    best_feature = sensor_df["best_model"].iloc[0]
    
    # Get full feature data for this sensor
    full_df = cached_feature_data[best_feature]
    full_df = full_df[full_df["sensor_id"] == sensor_id].copy()
    full_df = full_df.sort_values("date").reset_index(drop=True)
    
    # Get hindcast window
    cutoff_date = pd.Timestamp.now() - pd.DateOffset(months=HINDCAST_MONTHS)
    hindcast_df = full_df[full_df["date"] >= cutoff_date].copy()
    
    if hindcast_df.empty:
        continue
    
    # Get predictions from sensor_df
    predictions_merged = hindcast_df.merge(
        sensor_df[["date", "predicted_pm25"]], 
        on="date", 
        how="left"
    )
    
    # Prepare monitoring record with engineered features
    # Use .get() method which returns the column if it exists, otherwise creates a Series of NaN
    pred_df = predictions_merged[["date", "predicted_pm25"]].copy()
    pred_df["sensor_id"] = sensor_id
    pred_df["days_before_forecast_day"] = 0.0  # 0 for training/hindcast
    
    # Add engineered features if they exist in the source data
    pred_df["predicted_pm25_rolling_3d"] = predictions_merged.get("pm25_rolling_3d", pd.Series([np.nan] * len(predictions_merged)))
    pred_df["predicted_pm25_lag_1d"] = predictions_merged.get("pm25_lag_1d", pd.Series([np.nan] * len(predictions_merged)))
    pred_df["predicted_pm25_lag_2d"] = predictions_merged.get("pm25_lag_2d", pd.Series([np.nan] * len(predictions_merged)))
    pred_df["predicted_pm25_lag_3d"] = predictions_merged.get("pm25_lag_3d", pd.Series([np.nan] * len(predictions_merged)))
    pred_df["predicted_pm25_nearby_avg"] = predictions_merged.get("pm25_nearby_avg", pd.Series([np.nan] * len(predictions_merged)))
    
    monitoring_predictions.append(pred_df)
    # print(f"  Prepared {len(pred_df)} predictions for sensor {sensor_id}")
print(f"Prepared {len(monitoring_predictions)} sensors' predictions.")

# Combine all predictions
if monitoring_predictions:
    all_monitoring_df = pd.concat(monitoring_predictions, ignore_index=True)
    
    # Ensure correct data types to match feature group schema
    all_monitoring_df["date"] = pd.to_datetime(all_monitoring_df["date"])
    all_monitoring_df["sensor_id"] = all_monitoring_df["sensor_id"].astype(int)
    all_monitoring_df["predicted_pm25"] = all_monitoring_df["predicted_pm25"].astype(float)
    all_monitoring_df["days_before_forecast_day"] = all_monitoring_df["days_before_forecast_day"].astype(float)
    
    # Also convert the predicted feature columns to float
    for col in ["predicted_pm25_rolling_3d", "predicted_pm25_lag_1d", 
                "predicted_pm25_lag_2d", "predicted_pm25_lag_3d", "predicted_pm25_nearby_avg"]:
        if col in all_monitoring_df.columns:
            all_monitoring_df[col] = all_monitoring_df[col].astype(float)
    
    print(f"\n📊 Total predictions to insert: {len(all_monitoring_df)}")
    # print(f"\nSample data:")
    # print(all_monitoring_df.head())
    # print(f"\nData types:")
    # print(all_monitoring_df.dtypes)
    
    # Insert into feature group with retry logic
    print(f"\nInserting {len(all_monitoring_df)} predictions into aq_predictions feature group...")

    BATCH_SIZE = 2000
    MAX_RETRIES = 5
    INITIAL_BACKOFF = 2  # seconds

    num_batches = (len(all_monitoring_df) + BATCH_SIZE - 1) // BATCH_SIZE
    successful_batches = 0
    failed_batches = []

    for batch_idx, start in enumerate(range(0, len(all_monitoring_df), BATCH_SIZE), start=1):
        end = min(start + BATCH_SIZE, len(all_monitoring_df))
        batch = all_monitoring_df.iloc[start:end]
        
        success = False
        for attempt in range(1, MAX_RETRIES + 1):
            try:
                monitor_fg.insert(batch, write_options={"wait_for_job": False})
                success = True
                successful_batches += 1
                print(f"✅ Batch {batch_idx}/{num_batches} [{start}:{end}] inserted successfully")
                break
            except (ProtocolError, ConnectionError, Timeout, RequestException) as e:
                if attempt < MAX_RETRIES:
                    backoff_time = INITIAL_BACKOFF * (2 ** (attempt - 1))  # Exponential backoff
                    print(f"⚠️ Batch {batch_idx}/{num_batches} attempt {attempt}/{MAX_RETRIES} failed: {type(e).__name__}")
                    print(f"   Retrying in {backoff_time}s...")
                    time.sleep(backoff_time)
                else:
                    print(f"❌ Batch {batch_idx}/{num_batches} FAILED after {MAX_RETRIES} attempts")
                    failed_batches.append((batch_idx, start, end))
        
        # Small delay between batches to avoid overwhelming the server
        if success and batch_idx < num_batches:
            time.sleep(0.5)
    
    print(f"\n{'='*60}")
    print(f"✅ Successfully inserted {successful_batches}/{num_batches} batches")
    
    if failed_batches:
        print(f"❌ Failed batches: {len(failed_batches)}")
        for batch_idx, start, end in failed_batches:
            print(f"   Batch {batch_idx}: rows {start}-{end}")
    
    # Verify insertion
    print("\n🔍 Verifying insertion...")
    time.sleep(5)  # Wait longer for materialization
    
    verification_df = monitor_fg.read()
    print(f"✅ Feature group now contains {len(verification_df)} total rows")
    print(f"   Training predictions prepared: {len(all_monitoring_df)}")
    
else:
    print("⚠️ No predictions to insert")



Preparing predictions with features for 96 sensors...

Prepared 96 sensors' predictions.

📊 Total predictions to insert: 42810

Inserting 42810 predictions into aq_predictions feature group...


Uploading Dataframe: 100.00% |██████████| Rows 2000/2000 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: aq_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279184/jobs/named/aq_predictions_1_offline_fg_materialization/executions
✅ Batch 1/22 [0:2000] inserted successfully


Uploading Dataframe: 100.00% |██████████| Rows 2000/2000 | Elapsed Time: 00:01 | Remaining Time: 00:00


2026-01-28 10:09:19,823 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1279184/jobs/named/aq_predictions_1_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/kristina_titanic/Resources/jobs/aq_predictions_1_offline_fg_materialization/config_1769429691443) to trigger the materialization job again.

✅ Batch 2/22 [2000:4000] inserted successfully


...

✅ Batch 20/22 [38000:40000] inserted successfully


Uploading Dataframe: 100.00% |██████████| Rows 2000/2000 | Elapsed Time: 00:01 | Remaining Time: 00:00


✅ Batch 21/22 [40000:42000] inserted successfully


Uploading Dataframe: 100.00% |██████████| Rows 810/810 | Elapsed Time: 00:01 | Remaining Time: 00:00


✅ Batch 22/22 [42000:42810] inserted successfully

✅ Successfully inserted 22/22 batches

🔍 Verifying insertion...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.96s) 
✅ Feature group now contains 43544 total rows
   Training predictions prepared: 42810


In [50]:
all_test_data = []
total_sensors = len(best_models)

print(f"Generating visualizations for {total_sensors} sensors...\n")

for idx, (_, row) in enumerate(best_models.iterrows(), start=1):
    sensor_id = row.name
    best_feature = row["feature_name"]

    status = []

    # Load cached feature view data
    df = cached_feature_data[best_feature]
    df = df[df["sensor_id"] == sensor_id].copy()
    df = df.sort_values("date").reset_index(drop=True)
    df = df.dropna(subset=[TARGET])

    feature_cols = [c for c in df.columns if c not in EXCLUDE_COLS]

    if len(df) < MIN_ROWS:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (too few rows)")
        continue

    # Generate predictions for full dataset
    model_obj = models[best_feature][sensor_id]
    df["predicted_pm25"] = model_obj.predict(df[feature_cols])
    df["best_model"] = best_feature
    status.append("predictions")

    # Hindcast window (last N months)
    cutoff_date = pd.Timestamp.now() - pd.DateOffset(months=HINDCAST_MONTHS)
    df_hindcast = df[df["date"] >= cutoff_date].copy()

    if df_hindcast.empty:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (no recent data)")
        continue
    status.append("hindcast")

    # Attach metadata
    if sensor_id in aq_metadata.index:
        df["latitude"] = aq_metadata.at[sensor_id, "latitude"]
        df["longitude"] = aq_metadata.at[sensor_id, "longitude"]
        df["city"] = aq_metadata.at[sensor_id, "city"]
        df["street"] = aq_metadata.at[sensor_id, "street"]
        status.append("metadata")
    else:
        print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: skipped (no metadata)")
        continue

    # Save plots
    sensor_dir = f"{model_dir}/{sensor_id}"
    images_dir = f"{sensor_dir}/images"
    os.makedirs(images_dir, exist_ok=True)

    # Feature importance plot
    plot_importance(model_obj)
    plt.savefig(f"{images_dir}/feature_importance.png")
    plt.close()
    status.append("feature_importance")

    # Hindcast plot
    fig = visualization.plot_air_quality_forecast(
        df["city"].iloc[0],
        df["street"].iloc[0],
        df_hindcast,
        f"{images_dir}/hindcast_training.png",
        hindcast=True
    )
    if fig is not None:
        fig.suptitle(f"{df['city'].iloc[0]} {df['street'].iloc[0]} (Best Model: {best_feature})")
        plt.close(fig)
    status.append("hindcast_plot")

    # Append data
    all_test_data.append(
        df[[
            "date", "sensor_id", "pm25", "predicted_pm25",
            "latitude", "longitude", "city", "street", "best_model"
        ]]
    )
    status.append("saved")

    print(f"[{idx}/{total_sensors}] Sensor {sensor_id}: " + ", ".join(status))

print(f"\n✅ Visualization complete: {len(all_test_data)} sensors processed.")

Generating visualizations for 103 sensors...

[1/103] Sensor 57421: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[2/103] Sensor 58666: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[3/103] Sensor 58909: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[4/103] Sensor 58912: skipped (no recent data)
[5/103] Sensor 58921: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[6/103] Sensor 59095: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[7/103] Sensor 59356: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[8/103] Sensor 59410: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[9/103] Sensor 59497: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[10/103] Sensor 59593: predictions, hindcast, metadata, feature_importance, hindcast_plot, saved
[11/103] Sensor 59650: predictions, hindca

## 3.8. Model Registration

### 3.8.1. Create Training Dataset

In [51]:
# Create training datasets for each feature view
training_datasets = {}

MAX_RETRIES = 10  # Increased for quota issues
INITIAL_BACKOFF = 3
QUOTA_WAIT = 60  # Wait 60 seconds when hitting quota limits

print(f"Creating training datasets for {len(feature_views)} feature views...\n")

for idx, (feature_name, fv) in enumerate(feature_views.items(), start=1):
    success = False
    
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            # Create a training dataset version
            version, job = fv.create_training_data(
                description=f"Training data for {feature_name} features",
                data_format="csv",
                write_options={"wait_for_job": True}
            )
            training_datasets[feature_name] = version
            print(f"✅ [{idx}/{len(feature_views)}] {feature_name}: created training dataset v{version}")
            success = True
            break
            
        except RestAPIError as e:
            # Handle quota limits and other API errors
            error_msg = str(e).lower()
            if "quota" in error_msg or "parallel executions" in error_msg:
                if attempt < MAX_RETRIES:
                    print(f"⚠️ {feature_name}: attempt {attempt}/{MAX_RETRIES} - quota limit reached")
                    print(f"   Waiting {QUOTA_WAIT}s for jobs to complete...")
                    time.sleep(QUOTA_WAIT)
                else:
                    print(f"❌ {feature_name}: FAILED after {MAX_RETRIES} attempts (quota)")
                    raise
            else:
                if attempt < MAX_RETRIES:
                    backoff_time = INITIAL_BACKOFF * (2 ** (attempt - 1))
                    print(f"⚠️ {feature_name}: attempt {attempt}/{MAX_RETRIES} failed (RestAPIError)")
                    print(f"   Error: {str(e)[:150]}")
                    print(f"   Retrying in {backoff_time}s...")
                    time.sleep(backoff_time)
                else:
                    print(f"❌ {feature_name}: FAILED after {MAX_RETRIES} attempts")
                    raise
            
        except (ProtocolError, ConnectionError, Timeout, RequestException, OSError) as e:
            if attempt < MAX_RETRIES:
                backoff_time = INITIAL_BACKOFF * (2 ** (attempt - 1))
                print(f"⚠️ {feature_name}: attempt {attempt}/{MAX_RETRIES} failed ({type(e).__name__}: {str(e)[:100]})")
                print(f"   Retrying in {backoff_time}s...")
                time.sleep(backoff_time)
            else:
                print(f"❌ {feature_name}: FAILED after {MAX_RETRIES} attempts")
                raise
        
        except Exception as e:
            # Catch-all for unexpected errors
            if attempt < MAX_RETRIES:
                backoff_time = INITIAL_BACKOFF * (2 ** (attempt - 1))
                print(f"⚠️ {feature_name}: attempt {attempt}/{MAX_RETRIES} failed with unexpected error")
                print(f"   Error: {type(e).__name__}: {str(e)[:100]}")
                print(f"   Retrying in {backoff_time}s...")
                time.sleep(backoff_time)
            else:
                print(f"❌ {feature_name}: FAILED after {MAX_RETRIES} attempts with {type(e).__name__}")
                raise
    
    if not success:
        print(f"❌ Could not create training dataset for {feature_name}")
        raise RuntimeError(f"Failed to create training dataset for {feature_name}")
    
    # Small delay between feature views
    if idx < len(feature_views):
        time.sleep(2)

print(f"\n✅ All {len(training_datasets)} training datasets created successfully")


Creating training datasets for 7 feature views...

Training dataset job started successfully, you can follow the progress at 
http://c.app.hopsworks.ai/p/1279184/jobs/named/air_quality_baseline_fv_1_create_fv_td_28012026091324/executions
2026-01-28 10:13:32,865 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2026-01-28 10:13:39,213 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-28 10:19:03,380 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2026-01-28 10:19:03,552 INFO: Waiting for log aggregation to finish.
2026-01-28 10:19:12,191 INFO: Execution finished successfully.
2026-01-28 10:19:12,405 WARNING: VersionWarning: Incremented version to `35`.

✅ [1/7] baseline: created training dataset v35
Training dataset job started successfully, you can follow the progress at 
http://c.app.hopsworks.ai/p/1279184/jobs/named/air_quality_rolling_fv_1_create_fv_td_

### 3.8.2. Register Models

In [52]:
uploaded = 0
total_sensors = len(best_models)

print(f"Registering models for {total_sensors} sensors...\n")

MAX_RETRIES = 5
INITIAL_BACKOFF = 2

for sensor_id, row in best_models.iterrows():

    best_feature = row["feature_name"]
    best_r2 = row["R2"]
    best_mse = row["MSE"]

    # Get trained model
    model_obj = models[best_feature][sensor_id]

    # Save model locally
    sensor_model_dir = f"{model_dir}/{sensor_id}"
    os.makedirs(sensor_model_dir, exist_ok=True)
    model_obj.save_model(f"{sensor_model_dir}/model.json")

    # Register model
    model = mr.python.create_model(
        name=f"air_quality_xgboost_model_{sensor_id}",
        metrics={"R2": best_r2, "MSE": best_mse},
        feature_view=feature_views[best_feature],
        training_dataset_version=training_datasets[best_feature],
        description=f"PM2.5 predictor for sensor {sensor_id} using {best_feature} features",
    )

    success = False
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            model.save(sensor_model_dir)
            success = True
            break
        except (ProtocolError, ConnectionError, Timeout, RequestException) as e:
            if attempt < MAX_RETRIES:
                backoff_time = INITIAL_BACKOFF * (2 ** (attempt - 1))
                print(f"  ⚠️ Sensor {sensor_id}: attempt {attempt}/{MAX_RETRIES} failed ({type(e).__name__})")
                print(f"     Retrying in {backoff_time}s...")
                time.sleep(backoff_time)
            else:
                print(f"  ❌ Sensor {sensor_id}: FAILED after {MAX_RETRIES} attempts")

    if success:
        uploaded += 1
        print(f"[{uploaded}/{total_sensors}] Sensor {sensor_id}: registered ({best_feature})")
    else:
        print(f"[--/--] Sensor {sensor_id}: FAILED to register")

print(f"\n✅ Done. {uploaded}/{total_sensors} models successfully registered.")


Registering models for 103 sensors...



  0%|          | 0/6 [00:00<?, ?it/s]

Uploading c:\Users\krist\Documents\GitHub\pm25/models/57421/model.json: 0.000%|          | 0/459621 elapsed<00…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/57421\images/feature_importance.png: 0.000%|          | …

Uploading c:\Users\krist\Documents\GitHub\pm25/models/57421\images/forecast.png: 0.000%|          | 0/40744 el…

...

Uploading c:\Users\krist\Documents\GitHub\pm25/models/562600\images/forecast.png: 0.000%|          | 0/36866 e…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/562600\images/hindcast_prediction.png: 0.000%|          …

Uploading c:\Users\krist\Documents\GitHub\pm25/models/562600\images/hindcast_training.png: 0.000%|          | …

Uploading c:\Users\krist\Documents\GitHub\pm25\model_schema.json: 0.000%|          | 0/759 elapsed<00:00 remai…

Model created, explore it at https://c.app.hopsworks.ai:443/p/1279184/models/air_quality_xgboost_model_562600/18
[103/103] Sensor 562600: registered (lagged_1d)

✅ Done. 103/103 models successfully registered.


## 3.9. Upload Plots

In [53]:
dataset_api = project.get_dataset_api()
base_dir = "Resources/plots"
try:
    dataset_api.mkdir(base_dir)
except:
    pass

uploaded_images = 0
total_sensors = len(sensor_locations)

print(f"Uploading plots for {total_sensors} sensors...\n")

for sensor_id in sensor_locations.keys():
    sensor_dir = f"{base_dir}/{sensor_id}"
    try:
        dataset_api.mkdir(sensor_dir)
    except:
        pass

    local_path = f"{model_dir}/{sensor_id}/images/hindcast_training.png"
    remote_path = f"{sensor_dir}/hindcast_training.png"

    ok = hopsworks_admin.safe_upload(dataset_api, local_path, remote_path)

    if ok:
        uploaded_images += 1
        print(f"Uploaded image for sensor {sensor_id} ({uploaded_images}/{total_sensors})")
    else:
        print(f"❌ [fail] Sensor {sensor_id}: upload failed after retries")

print(f"✅ Done uploading {uploaded_images} images.")


Uploading plots for 103 sensors...



Uploading c:\Users\krist\Documents\GitHub\pm25/models/60853/images/hindcast_training.png: 0.000%|          | 0…

Uploaded image for sensor 60853 (1/103)


Uploading c:\Users\krist\Documents\GitHub\pm25/models/59497/images/hindcast_training.png: 0.000%|          | 0…

Uploaded image for sensor 59497 (2/103)


...

Uploaded image for sensor 401314 (101/103)


Uploading c:\Users\krist\Documents\GitHub\pm25/models/562600/images/hindcast_training.png: 0.000%|          | …

Uploaded image for sensor 562600 (102/103)


Uploading c:\Users\krist\Documents\GitHub\pm25/models/556792/images/hindcast_training.png: 0.000%|          | …

Uploaded image for sensor 556792 (103/103)
✅ Done uploading 103 images.
